In [25]:
import os
from google.cloud import speech
import io
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\client_service_key.json'
client = speech.SpeechClient()

In [11]:
bucket_name = 'speech-to-text1705'
Name = 'Audio_wav'

In [26]:
from google.cloud import storage
def list_buckets():
    storage_client = storage.Client()
    buckets = storage_client.list_buckets()
    
    for bucket in buckets:
        print(bucket.name)
        from google.cloud import storage
        
def create_bucket_class_location(bucket_name):
    """
    Create a new bucket in the US region with the coldline storage
    class
    """
    # bucket_name = "your-new-bucket-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "COLDLINE"
    new_bucket = storage_client.create_bucket(bucket, location="us")

    print(
        "Created bucket {} in {} with storage class {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )
    return new_bucket

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [27]:
def Config_GGC(sample_rate_hertz=44100,
              audio_channel_count=1,
              model=None,
              enable_automatic_punctuation=True):
    if model != None:
        config_wav_enhanced = speech.RecognitionConfig(
        sample_rate_hertz = sample_rate_hertz,
        enable_automatic_punctuation = enable_automatic_punctuation,
        language_code = 'vi-VN',
        audio_channel_count = audio_channel_count,
        model = model,
        )
    else:
        config_wav_enhanced = speech.RecognitionConfig(
        sample_rate_hertz = sample_rate_hertz,
        enable_automatic_punctuation=True,
        language_code = 'vi-VN',
        audio_channel_count=audio_channel_count
        )
    return config_wav_enhanced

def Transribe_Long_Audio(Audio_wav,config_wav_enhanced,
                        bucket_name = 'speech-to-text1705',
                        Name = 'Audio_wav'):
    client = speech.SpeechClient()
    Audio_name = Audio_wav.split('/')[-1].split('.')[0]
    upload_blob(bucket_name,Audio_wav,Name)
    media_uri = 'gs://{}/{}'.format(bucket_name, Name)
    long_audi_wav = speech.RecognitionAudio(uri=media_uri)
    
    operations = client.long_running_recognize(
        config = config_wav_enhanced,
        audio = long_audi_wav
    )
    response = operations.result(timeout=90)
    text = []
    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        text.append(alternative.transcript + '\n')
    return text

def Transcribe_Short_Audio(Audio_wav,config_wav_enhanced):
    client = speech.SpeechClient()
    with io.open(Audio_wav, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    # print(type(audio))
    
    response = client.recognize(config=config_wav_enhanced, audio=audio)
    text = []
    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
#         print("-" * 20)
#         print("First alternative of result {}".format(i))
#         print("Transcript: {}".format(alternative.transcript))
        text.append(alternative.transcript + '\n')
    return text

In [14]:
os.getcwd()
os.chdir('C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\Result')

In [22]:
path = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\Result\\Audio'

In [16]:
for root, dirs, files in os.walk(path, topdown=False):
    audio_list_file = files
audio_list_file = audio_list_file[0:5]

In [17]:
Result_txt = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\Result\\result.txt'

In [28]:
os.chdir(path)
transcript = []
for i in range(len(audio_list_file)):
    Audio_wav = audio_list_file[i]
    config = Config_GGC(sample_rate_hertz = 48000,
                    audio_channel_count = 2)
    text = Transcribe_Short_Audio(Audio_wav,config)
    with open(Result_txt, 'w+',encoding='utf-8') as f:
        f.writelines(text)
        f.write('\n')

In [29]:
def Transcribe_Audios(Folder, config, Result_txt):
    audios = os.listdir(Folder)
    audios = [os.path.join(Folder, i) for i in audios]
    
    with open(Result_txt, 'w+', encoding='utf-8') as f:
        for audio in audios:
            text = Transcribe_Short_Audio(audio, config)
            f.writelines(text)
            f.write('\n')
    config = Config_GGC(sample_rate_hertz=48000,
                       audio_channel_count=2)
    

In [20]:
Result_txt = 'C:\\Users\\quang\\Data Analysis\\STECH\\Speech-To-Text\\Speech-Recognition\\Multiple_speaker_detection\\Result\\result.txt'

In [21]:
Transcribe_Audios(path, config, Result_txt)

InvalidArgument: 400 Invalid recognition 'config': bad encoding..